In [6]:

import pandas as pd
import json
import nlp

train_checkworthy = pd.read_csv("data/CT24_checkworthy_english/CT24_checkworthy_english_train.tsv", sep='\t')
#print(train_checkworthy)

with open("data/English_train.json") as json_file:
    json_strs = json_file.readlines()

json_datas = [json.loads(json_str) for json_str in json_strs]

train_stance = pd.DataFrame.from_dict(json_datas)
#print(train_stance)

In [5]:
test = json_datas[0]
test1 = json_datas[2]

In [20]:
datasets = {
    "cw-train": nlp.load_dataset("csv", name="cw-train", data_files="data/checkworthy/CT24_checkworthy_english_train.tsv", delimiter="\t"),
    "cw-dev": nlp.load_dataset("csv", name="cw-dev", data_files="data/checkworthy/CT24_checkworthy_english_dev.tsv", delimiter="\t"),
    "cw-dev-test": nlp.load_dataset("csv", name="cw-dev-test", data_files="data/checkworthy/CT24_checkworthy_english_dev-test.tsv", delimiter="\t"),

        }

Using custom data configuration cw-train
Using custom data configuration cw-dev
Using custom data configuration cw-dev-test


In [22]:
for task_name, dataset in datasets.items():
    print(task_name)
    print(datasets[task_name]["train"][1])
    print()

cw-train
{'Sentence_id': 19099, 'Text': "Now, let's balance the budget and protect Medicare, Medicaid, education and the environment.", 'class_label': 'No'}

cw-dev
{'Sentence_id': 80, 'Text': "We're consuming 50 percent of the world's cocaine.", 'class_label': 'Yes'}

cw-dev-test
{'Sentence_id': 37440, 'Text': "There's no way they would give it up.", 'class_label': 'No'}



In [41]:
#https://colab.research.google.com/github/zphang/zphang.github.io/blob/master/files/notebooks/Multi_task_Training_with_Transformers_NLP.ipynb#scrollTo=aVX5hFlzmLka
import transformers
import torch.nn as nn
class MultiTask(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        super().__init__(transformers.PretrainedConfig())
        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_conf_dict):
        shared_encoder = None
        taskmodels_dict = {}

        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name,
                config=model_conf_dict[task_name]
            )

            if shared_encoder is None:
                shared_encoder = getattr(model, cls.get_encoder_attr_name(model))
            else:
                setattr(model, cls.get_encoder_attr_name(model), shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)
        
    @classmethod
    def get_encoder_attr_name(cls, model):
        model_class_name = model.__class__.__name__
        if model_class_name.startswith("Bert"):
            return "bert"
        elif model_class_name.startswith("Roberta"):
            return "roberta"

        else:
            raise KeyError(f"Add support for new model {model_class_name}")
        
    def forward(self, task_name, **kwargs):
        return self.taskmodel_dict[task_name](**kwargs)
#test = MultiTask(transformers.BertModel.from_pretrained("bert-base-uncased"), datasets)

In [42]:
model_name = "roberta-base"

model_type_dict={
        "cw-train": transformers.AutoModelForSequenceClassification,
        "cw-dev": transformers.AutoModelForSequenceClassification,

    }
model_config_dict={
        "cw-train": transformers.AutoConfig.from_pretrained(model_name, num_labels=1),
        "cw-dev": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),

    }

multitask_model = MultiTask.create(
    model_name,
    model_type_dict,
    model_config_dict
    
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
if model_name.startswith("roberta"):
    print(multitask_model.encoder.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["cw-train"].roberta.embeddings.word_embeddings.weight.data_ptr())
    print(multitask_model.taskmodels_dict["cw-dev"].roberta.embeddings.word_embeddings.weight.data_ptr())
else:
    print("Exercise for the reader: add a check for other model architectures =)")

139656049266752
139656049266752
139656049266752
